In [1]:
experiment_name = "news_exp"

In [2]:
initial_prompt = """
You goal is to classify the following news headlines into one of the following categories:
1- World
2- Sports
3- Business
4- Sci/Tech

Provide your response as a JSON dictionary with the following structure:
{
    "chain_of_thought": "Your step-by-step reasoning here",
    "topic": "The identified topic - the ONLY possible topics are "World", "Sports", "Business", "Sci/Tech". Output just one single category."
}
"""

In [3]:
# Output format prompt
output_format_prompt = """
Provide your response as a JSON dictionary with the following structure:
{
    "chain_of_thought": "Your step-by-step reasoning here",
    "topic": "The identified topic - the ONLY possible topics are "World", "Sports", "Business", "Sci/Tech". Output just one single category."
}
"""

In [4]:
# Define output schema
output_schema = {
    'key_to_extract': 'topic',
    'value_mapping': {
        'World': 'World',
        'Sports': 'Sports',
        'Business': 'Business',
        'Sci/Tech': 'Sci/Tech',
    },
    'regex_pattern': r'"topic":\s*"([^"]+)"',  # Changed from \w+ to [^"]+ to capture everything until the closing quote
    #
    'chain_of_thought_key': 'chain_of_thought',  
    'chain_of_thought_regex': r'"chain_of_thought":\s*"(.*?)"',
    #
    'use_json_mode': True,
}

In [5]:
# Set number of optimization iterations
iterations = 5

In [6]:
# Define model providers and models for evaluation and optimization
eval_provider = "openai"
eval_model = "gpt-4o-mini"
optim_provider = "openai"
optim_model = "gpt-4o-mini"

In [7]:
# Path to the CSV file containing review data for evaluation
eval_datapath = "news.csv"
text_column = "Title"
target_column = "topic"
sample_size = 10

------------------------------------------------------------------------------------------

In [8]:
# Import necessary libraries
import pandas as pd
import sys
import os
# Add the parent directory to sys.path
# Use getcwd() to get the current working directory for Jupyter notebooks
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
grandparent_dir = os.path.dirname(parent_dir)
sys.path.append(grandparent_dir)
from src import optimize_prompt

In [9]:
# Load and prepare data
eval_data = pd.read_csv(eval_datapath, encoding='ISO-8859-1', usecols=[text_column, target_column])
eval_data = eval_data.rename(columns={text_column: 'text', target_column: 'label'})
eval_data["text"] = eval_data["text"].astype(str)
eval_data

,text,label
0,Fears for T N pension after talks,Business
1,The Race is On: Second Private Team Sets Launc...,Sci/Tech
2,Ky. Company Wins Grant to Study Peptides (AP),Sci/Tech
3,Prediction Unit Helps Forecast Wildfires (AP),Sci/Tech
4,Calif. Aims to Limit Farm-Related Smog (AP),Sci/Tech
...,...,...
7595,Around the world,World
7596,Void is filled with Clement,Sports
7597,Martinez leaves bitter,Sports
7598,5 of arthritis patients in Singapore take Bext...,Business


In [10]:
# Randomly select n samples from each class
eval_data = (
    eval_data.groupby('label')
    .apply(lambda x: x.sample(n=sample_size, random_state=42))
    .reset_index(drop=True)
)
# Shuffle the DataFrame randomly
eval_data = eval_data.sample(frac=1, random_state=42).reset_index(drop=True)
print(f"Evaluation data shape: {eval_data.shape}")
print(eval_data.head())

Evaluation data shape: (40, 2)
                                                text     label
0         Greenspan: Debt, home prices not dangerous  Sci/Tech
1  Explorers Find Ancient City in Remote Peru Jun...  Sci/Tech
2                           Countdown to Deep Impact  Sci/Tech
3                     Ali gives Iraq fighting chance    Sports
4  Lloyds TSB to Move More Than 1,000 UK Jobs to ...  Business


In [11]:
# Run the prompt optimization process
best_prompt, best_metrics = optimize_prompt(
    initial_prompt = initial_prompt,
    eval_data = eval_data,
    iterations =iterations,
    eval_provider=eval_provider,
    eval_model=eval_model,
    eval_temperature=0.7,
    optim_provider=optim_provider,
    optim_model=optim_model,
    optim_temperature=0,
    use_cache=True,
    output_format_prompt = output_format_prompt,
    output_schema=output_schema,
    fp_comments = "",
    fn_comments = "",
    tp_comments = "",
    invalid_comments="",
    validation_comments="",
    experiment_name = experiment_name,
    skip_prompt_validation = True,
)


Detected problem type: multiclass
Selected evaluation provider: openai
Selected evaluation model: gpt-4o-mini
Evaluation temperature: 0.7
Selected optimization provider: openai
Selected optimization model: gpt-4o-mini
Optimization temperature: 0
Estimated token usage: 121600
Estimated cost: $0.15

Do you want to proceed with the optimization? (Y/N): 
Iteration 1/5


╭────────────────────────────────────────────── Current Full Prompt ──────────────────────────────────────────────╮
│                                                                                                                 │
│ You goal is to classify the following news headlines into one of the following categories:                      │
│ 1- World                                                                                                        │
│ 2- Sports                                                                                                       │
│ 3- Business                                                                                                     │
│ 4- Sci/Tech                                                                                                     │
│                                                                                                                 │
│ Provide your response as a JSON dictionary with the following structure:                                        │
│ {                                                                                                               │
│     "chain_of_thought": "Your step-by-step reasoning here",                                                     │
│     "topic": "The identified topic - the ONLY possible topics are "World", "Sports", "Business", "Sci/Tech".    │
│ Output just one single category."                                                                               │
│ }                                                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

-----------------------------------
Processing text 1/40 .....
Using cached output for text 1/40
Prediction 1/40: Business | Ground Truth: Sci/Tech ❌ (Incorrect)
-----------------------------------
Processing text 2/40 .....
Using cached output for text 2/40
Prediction 2/40: Sci/Tech | Ground Truth: Sci/Tech ✅ (Correct)
-----------------------------------
Processing text 3/40 .....
Using cached output for text 3/40
Prediction 3/40: Sci/Tech | Ground Truth: Sci/Tech ✅ (Correct)
-----------------------------------
Processing text 4/40 .....
Using cached output for text 4/40
Prediction 4/40: World | Ground Truth: Sports ❌ (Incorrect)
-----------------------------------
Processing text 5/40 .....
Using cached output for text 5/40
Prediction 5/40: Business | Ground Truth: Business ✅ (Correct)
-----------------------------------
Processing text 6/40 .....
Using cached output for text 6/40
Prediction 6/40: Sci/Tech | Ground Truth: Sci/Tech ✅ (Correct)
-----------------------------------
Proce

Evaluation Metrics - Iteration 1
┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃ Metric              ┃  Value ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ Precision           │ 0.8920 │
│ Recall              │ 0.8750 │
│ Accuracy            │ 0.8750 │
│ F1-score            │ 0.8779 │
│                     │        │
├─────────────────────┼────────┤
│ Valid Predictions   │     40 │
│ Invalid Predictions │      0 │
└─────────────────────┴────────┘

       Confusion Matrix        
┏━━━━━━━━━━━━━┳━━━┳━━━┳━━━┳━━━┓
┃ True↓/Pred→ ┃ 0 ┃ 1 ┃ 2 ┃ 3 ┃
┡━━━━━━━━━━━━━╇━━━╇━━━╇━━━╇━━━┩
│ 0           │ 9 │ 0 │ 0 │ 1 │
├─────────────┼───┼───┼───┼───┤
│ 1           │ 1 │ 9 │ 0 │ 0 │
├─────────────┼───┼───┼───┼───┤
│ 2           │ 0 │ 0 │ 8 │ 2 │
├─────────────┼───┼───┼───┼───┤
│ 3           │ 1 │ 0 │ 0 │ 9 │
└─────────────┴───┴───┴───┴───┘

...

## Improved Prompt:

Your goal is to classify the following news headlines into one of the following categories:
1- World
2- Sports
3- Business
4- Sci/Tech

When classifying, adhere to the following guidelines:

- **World**: Look for geopolitical events, conflicts, or significant social issues. Avoid classifying headlines as 
'World' if they primarily discuss competitive events or sports terminology. Be cautious of geographical references 
that may relate to sports or competition.

- **Sports**: Identify terms related to teams, players, or specific sports terminology. If a headline includes 
geographical mentions but focuses on competition (e.g., "fighting chance"), classify it as 'Sports' rather than 
'World'. Do not classify headlines as 'Sports' if they discuss economic or technological implications.

- **Business**: Focus on financial terminology, corporate actions, and economic indicators. If a headline discusses
economic issues with geopolitical implications (e.g., oil exports), classify it as 'World' instead of 'Business'. 
Avoid classifying headlines as 'Business' if they primarily discuss technological advancements.

- **Sci/Tech**: Look for terms related to technology, scientific exploration, or digital advancements. If economic 
discussions are framed in a technological context, classify them as 'Sci/Tech' rather than 'Business'. Do not 
classify headlines as 'Sci/Tech' if they focus solely on geopolitical or competitive aspects.

**Negative Instructions**:
- Do not classify headlines as 'World' if they contain explicit sports terminology.
- Avoid classifying as 'Business' when the headline has strong geopolitical implications.
- Refrain from classifying as 'Sports' if the headline lacks clear competitive language.

Provide your response as a JSON dictionary with the following structure:
{
    "chain_of_thought": "Your step-by-step reasoning here, breaking down the headline into its components and 
assessing the implications of each term. Ensure your reasoning avoids overgeneralization and aligns closely with 
the specific context of the headline.",
    "topic": "The identified topic - the ONLY possible topics are 'World', 'Sports', 'Business', 'Sci/Tech'. Output
just one single category."
}

╭────────────────╮
│ Output Format: │
╰────────────────╯

Provide your response as a JSON dictionary with the following structure:
{
    "chain_of_thought": "Your step-by-step reasoning here",
    "topic": "The identified topic - the ONLY possible topics are "World", "Sports", "Business", "Sci/Tech". Output
just one single category."
}

                                   Comparison of All Iterations                                   
┏━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃ Iteration ┃ Precision ┃ Recall ┃ Accuracy ┃ F1-score ┃ Valid Predictions ┃ Invalid Predictions ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│     1     │    0.8920 │ 0.8750 │   0.8750 │   0.8779 │           40.0000 │              0.0000 │
│     2     │    0.8920 │ 0.8750 │   0.8750 │   0.8779 │           40.0000 │              0.0000 │
│     3     │    0.9333 │ 0.9250 │   0.9250 │   0.9245 │           40.0000 │              0.0000 │
│     4     │    0.9091 │ 0.9000 │   0.9000 │   0.9023 │           40.0000 │              0.0000 │
│     5     │    0.9333 │ 0.9250 │   0.9250 │   0.9260 │           40.0000 │              0.0000 │
└───────────┴───────────┴────────┴──────────┴──────────┴───────────────────┴─────────────────────┘


All logs saved in directory: /Users/danielfiuzadosil/Documents/GitHub/AI-Prompt-Optimiser/examples/muticlass_classifier/runs/news_exp_Wed_11-Dec-2024_11-30-46
